In [4]:
import panel as pn
pn.extension('tabulator', template='bootstrap', sizing_mode='stretch_width')

import pyvo as vo
import datetime as dt
from astropy.time import Time
import param
from pypika import Table, Criterion, EmptyCriterion
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u
import warnings
from astroquery.exceptions import TableParseError

# This is still a very basic program and basically the only thing that it does is talk to the Kapteyn archive
# It shows the entries in the database for the specific filters that the user has specified
# For now users can specify a date range, what kind of columns they want and whether the right ascension and declination should be null
# They can then select entries from the generated table and view that data on another tab
# For now users cannot do anything with that data, but these functions will be added
# I will add the ability to plot, specify more filters and add some basic functions, such as aligning images
# I will also add the ability to search for a specific object and using SIMBAD fetch the coordinates and find corresponding entries in the archive
# I will also add more comments and docstrings, once everything is a bit better laid out
# If you have any suggestions on what you would like to see added, please let me know
# Also, if you see any code that could be better, let me know as well

In [5]:
# Make connection to the TAP service
url = "https://vo.astro.rug.nl/tap"
service = vo.dal.TAPService(url)

In [6]:
# To create the queries I use PyPika, which is a query builder
# It supports several SQL dialects, but not ADQL, so I added some very basic ADQL functioning
# These classes override the existing PyPika classes
# With this it should also be relatively easy to add more custom functionality especially things like cone searches

from pypika.queries import Query, QueryBuilder
from pypika.utils import builder, QueryException
from typing import Any, Optional
from enum import Enum

class Dialects(Enum):
    ADQL = 'adql'

class ADQLQuery(Query):
    @classmethod
    def _builder(cls, **kwargs: Any) -> "QueryBuilder":
        return ADQLQueryBuilder(**kwargs)

class ADQLQueryBuilder(QueryBuilder):
    QUOTE_CHAR = None
    ALIAS_QUOTE_CHAR = '"'
    QUERY_ALIAS_QUOTE_CHAR = ''
    QUERY_CLS = ADQLQuery

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(dialect=Dialects.ADQL, **kwargs)
        self._contains = None
        self._top = None

    @builder
    def top(self, value):
        try:
            self._top = int(value)
        except ValueError:
            raise QueryException('TOP value must be an integer')

    def get_sql(self, *args, **kwargs):
        return super(ADQLQueryBuilder, self).get_sql(*args, groupby_alias=False, **kwargs)

    def _top_sql(self):
        if self._top:
            return 'TOP {} '.format(self._top)
        else:
            return ''

    @builder
    def contains(self, from_table, ra, dec, cone_radius):
        self._contains = [from_table, ra, dec, cone_radius]

    def _contains_sql(self):
        if self._contains:
            return "CONTAINS(POINT('ICRS', {}.ra, {}.dec), CIRCLE('ICRS', {}, {}, {}))=1 AND ".format(
                self._contains[0], self._contains[0], self._contains[1], self._contains[2], self._contains[3])
        else:
            return ''

    def _select_sql(self, **kwargs: Any) -> str:
        return "SELECT {distinct}{top}{select}".format(
            top=self._top_sql(),
            distinct="DISTINCT " if self._distinct else "",
            select=",".join(term.get_sql(with_alias=True, subquery=True, **kwargs) for term in self._selects)
        )

    @builder
    def _where_sql(self, quote_char: Optional[str] = None, **kwargs: Any) -> str:
        return " WHERE {contains}{where}".format(
            contains=self._contains_sql(),
            where=self._wheres.get_sql(quote_char=quote_char, subquery=True, **kwargs)
        )

In [7]:
# This class creates the widgets the user can use and creates all the little parts needed for the query
options = {'Filename': 'filename',
            'Observation date': 'kw_date_obs',
            'Image type': 'kw_imagetyp',
            'Right ascension': 'ra',
            'Declination': 'dec',
            'Object': 'kw_object'}
class CompileQuery(param.Parameterized):
    dates = param.DateRange((dt.datetime(2008, 5, 20, 12), dt.datetime(2022, 5, 1, 12)))
    select_columns = param.ListSelector(default=['Filename', 'Observation date', 'Right ascension', 'Declination', 'Object'], objects=list(options.keys()))
    ra_dec_not_null = param.Boolean(default=True)
    select_object = param.String(default='')
    cone_radius = param.Number(default=0.1, bounds=(0.0, 3.0))

    def table_string(self):
        table_name = 'observations.raw'
        return table_name

    def from_query(self):
        raw_observations = Table(self.table_string())
        return raw_observations

    @param.depends('select_columns', watch=True)
    def select_query(self):
        select_options = ''
        for i in self.select_columns:
            select_options += str(self.from_query()).strip('"') + '.' + options[i] + ','
        return select_options.rstrip(',')

    @param.depends('select_object', watch=True)
    def find_object(self):
        if self.select_object == '':
            return True
        else:
            object_name = self.select_object
            with warnings.catch_warnings():
                warnings.filterwarnings('error')
                try:
                    simbad_object = Simbad.query_object(object_name)
                    coordinates = SkyCoord(simbad_object['RA'], simbad_object['DEC'], unit=(u.hourangle, u.deg))
                    object_ra, object_dec = coordinates.ra.deg[0], coordinates.dec.deg[0]
                    return object_ra, object_dec
                except (Warning, TableParseError):
                    return False

    @param.depends('ra_dec_not_null', watch=True)
    def not_null_check(self):
        if self.ra_dec_not_null:
            return True

    @param.depends('dates', watch=True)
    def date_query(self):
        start_date = Time(self.dates[0]).jd
        end_date = Time(self.dates[1]).jd
        return start_date, end_date

In [8]:
# This class fetches the data using all the little parts of the query given by the CompileQuery class
# It returns a pandas dataframe, which can be displayed
class FetchData(CompileQuery):
    action = param.Action(lambda x: x.param.trigger('action'), label='Search')

    @param.depends('action')
    def data(self):
        if not self.find_object():
            return {'type': 'danger', 'message': "The object " + self.select_object +  " could not be found."}
        observation_table = self.from_query()
        dates = self.date_query()
        query = ADQLQuery.from_(
            observation_table
        ).select(
            self.select_query()
        ).where(
            Criterion.all([
                observation_table.ra.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.dec.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.obs_jd[dates[0]:dates[1]]
            ])
        ).orderby(observation_table.obs_jd).top(100)
        if self.select_object != '':
            query = query.contains(self.table_string(), self.find_object()[0], self.find_object()[1], self.cone_radius)
        result = service.search(query.get_sql())
        data_pandas = result.to_table().to_pandas()
        return data_pandas

In [9]:
# This class creates the responsive tables
# Whenever a user updates one of the filtering options, this will generate the new table
# This also makes sure that whenever a user selects an entry, it gets picked up and can be used
class CreateTable(FetchData):
    table = param.DataFrame(precedence=-1)
    table.default = FetchData().data()

    def __init__(self, **params):
        super().__init__(**params)
        self.table_widget = pn.widgets.Tabulator(self.table,
                                                 disabled=True,
                                                 pagination='local',
                                                 show_index=False,
                                                 layout='fit_columns',
                                                 selectable='checkbox-single',
                                                 text_align='left',
                                                 min_height=500)

    @param.depends('data', watch=True)
    def full_table(self):
        self.table_widget.value = self.data()

    @param.depends('table_widget.selected_dataframe')
    def selected_table(self):
        return pn.widgets.Tabulator(self.table_widget.selected_dataframe)

In [10]:
# This class puts the widgets and the tables together to create the UI
class CreateView(CreateTable):
    def panel(self):
        widgets = {
            'dates': pn.widgets.DatetimeRangePicker,
            'select_columns': {'widget_type': pn.widgets.CrossSelector, 'definition_order': False},
            'ra_dec_not_null': pn.widgets.Checkbox,
            'cone_radius': {'widget_type': pn.widgets.FloatSlider, 'name': 'Cone radius (degrees)' , 'step': 0.01},
            'action': {'widget_type': pn.widgets.Button, 'button_type': 'primary'}
        }
        settings = pn.Row(
            pn.Param(self, widgets=widgets, width=300, sizing_mode="fixed", name="Settings")
        )
        bootstrap.sidebar.append(settings)
        tabs = pn.Tabs(
            ('Data Table', self.table_widget),
            ('Data Plotting', pn.Card(self.selected_table))
        )
        bootstrap.main.append(tabs)
        return bootstrap

In [11]:
# Here I create an instance of the CreateView class and run the program
# It should automatically open a new tab in your browser
bootstrap = pn.template.BootstrapTemplate(title='Bootstrap Template')

dashboard = CreateView()
dashboard.panel().show();

Launching server at http://localhost:53384
